In [5]:
from bs4 import BeautifulSoup
import requests
import datetime
import pandas as pd
import re #regular expression
from collections import defaultdict # defaultdict: store a list with each key


In [6]:
input_skill = input('Enter your skill: ').strip()
place = input('Enter preferred location: ').strip()
no_of_pages = int(input('Enter # number of pages to scrape: '))

indeed_post = []

for page in range(no_of_pages):
    url = 'https://www.indeed.com/jobs?q=data%20analyst&l=US&fromage=1&vjk=91152078d1e41826' + input_skill + \
    '&1=' + place +'&sort=date' + '&start=' + str(page*10)
    
#request indeed url    
    html_content = requests.get(url)
    
#parse 
    soup = BeautifulSoup(html_content.text, 'lxml')
    outer_most_body = soup.find('div', attrs={'id':'mosaic-provider-jobcards'})

#locating the tag holding all job titles
    for i in outer_most_body.find('ul'):
        job_title = i.find('h2', {'class':'jobTitle jobTitle-newJob css-bdjp2m eu4oa1w0'})
        
#extracting the span tag holding job title
        if job_title != None:
            job = job_title.find('span').text
            
#extracting class holding company name
            if i.find('span', {'class':'companyName'}) != None:
                company = i.find('span', {'class':'companyName'}).text
            
#extracting the a tag holding the job link    
                if i.find('a') != None:
                    link = i.find('a', {'class':'jcs-JobTitle css-jspxzf eu4oa1w0'})['href']
                    
#extracting the class holding salary if specified
                    if i.find('div',{'class':'metadata estimated-salary-container'}) != None:
                        salary = i.find('div', {'class':'metadata estimated-salary-container'}).text
                    else:
                        salary = 'No salary'
                        
#extracting the span tag holding the job posted date
                        if i.find('span',{'class':'date'}) != None:
                            post_date = i.find('span', {'class':'date'}).text
                            
#append all extracted tags object into existing 'indeed_post' list created at the very top
                    indeed_post.append([job, company, link, salary, post_date])

indeed_dict_list=defaultdict(list)
indeed_spec = ['Jobs', 'Companys', 'Links', 'Salaries', 'Job_posted_date']
indeed_post

Enter your skill: data analyst
Enter preferred location: US
Enter # number of pages to scrape: 2


[['Data Analyst, Customer Success',
  'Square',
  '/rc/clk?jk=2d45ba3e438088bf&fccid=08bfce1064449178&vjs=3',
  'Estimated $75.3K - $95.3K a year',
  'PostedJust posted'],
 ['Sr Data Analyst',
  'Bank of America',
  '/rc/clk?jk=57fa6ae7a1e0a7e0&fccid=5bd99dfa21c8a490&vjs=3',
  'No salary',
  'PostedJust posted'],
 ['Data Analyst',
  'Infoobjects',
  '/rc/clk?jk=3a15e45e773fa19c&fccid=97aeac9e13420656&vjs=3',
  'Estimated $56.6K - $71.6K a year',
  'PostedJust posted'],
 ['Data Analyst I/II',
  'Summit Utilities, Inc.',
  '/rc/clk?jk=49f70a3d7d93492a&fccid=4b77d6308c3b533f&vjs=3',
  'Estimated $66.7K - $84.5K a year',
  'PostedJust posted'],
 ['Data Analyst Advisor',
  'General Dynamics Information Technology',
  '/rc/clk?jk=ae05ab5ec5b58c95&fccid=11caadcdc98800d4&vjs=3',
  'No salary',
  'PostedJust posted'],
 ['Part-Time Data Entry Analyst - Solvas Financial Technology',
  'Deloitte',
  '/rc/clk?jk=0d68ea00752a9d7a&fccid=9e215d88a6b33622&vjs=3',
  'No salary',
  'PostedJust posted'],


In [9]:
indeed_jobs_df = pd.DataFrame(indeed_post, columns=indeed_spec)
# Fixing posting date using regular expression
date_converted=[]

for i in indeed_jobs_df['Job_posted_date']:
    if re.findall(r'[0-9]',i):
        
#if the string has digits convert each entry to single strings: ['3', '0']->'30'
        convert_exp_single_string = ''.join(re.findall(r'[0-9]',i))
    
#convert string int to int and subtract from today's date and format
        convert_date_2_int = (datetime.datetime.today() - datetime.timedelta(int(convert_exp_single_string))).strftime('%Y-%m-&d')
        date_converted.append(convert_date_2_int )
    else:
        date_converted.append(datetime.datetime.today().strftime('%Y-%m-%d'))

In [10]:
indeed_jobs_df['Job_posted_date:']=date_converted
indeed_jobs_df

,Jobs,Companys,Links,Salaries,Job_posted_date,Job_posted_date:
0,"Data Analyst, Customer Success",Square,/rc/clk?jk=2d45ba3e438088bf&fccid=08bfce106444...,Estimated $75.3K - $95.3K a year,PostedJust posted,2022-07-22
1,Sr Data Analyst,Bank of America,/rc/clk?jk=57fa6ae7a1e0a7e0&fccid=5bd99dfa21c8...,No salary,PostedJust posted,2022-07-22
2,Data Analyst,Infoobjects,/rc/clk?jk=3a15e45e773fa19c&fccid=97aeac9e1342...,Estimated $56.6K - $71.6K a year,PostedJust posted,2022-07-22
3,Data Analyst I/II,"Summit Utilities, Inc.",/rc/clk?jk=49f70a3d7d93492a&fccid=4b77d6308c3b...,Estimated $66.7K - $84.5K a year,PostedJust posted,2022-07-22
4,Data Analyst Advisor,General Dynamics Information Technology,/rc/clk?jk=ae05ab5ec5b58c95&fccid=11caadcdc988...,No salary,PostedJust posted,2022-07-22
5,Part-Time Data Entry Analyst - Solvas Financia...,Deloitte,/rc/clk?jk=0d68ea00752a9d7a&fccid=9e215d88a6b3...,No salary,PostedJust posted,2022-07-22
6,Senior Data Analyst,US Federal Trade Commission,/rc/clk?jk=bb66a72d1b802ac0&fccid=dd616958bd9d...,No salary,PostedJust posted,2022-07-22
7,Data Analyst Stf,LOCKHEED MARTIN CORPORATION,/rc/clk?jk=0a1b427fcd131052&fccid=aeb15e43a680...,No salary,PostedJust posted,2022-07-22
8,Data Analyst,RIITHINK DIGITAL MARKETING,/rc/clk?jk=548746be84f19c76&fccid=807237cbbfe5...,Estimated $69.1K - $87.5K a year,PostedJust posted,2022-07-22
9,Quality Assurance Data Analyst,ISO,/rc/clk?jk=7ad49bc695281740&fccid=d621f9a5603c...,Estimated $58K - $73.4K a year,PostedJust posted,2022-07-22
